# Libraries

In [1]:
import  pandas as pd
import  numpy as np
import  json
import  plotly.express as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go
from    xgboost         import XGBClassifier
from    sklearn.metrics import accuracy_score
from    sklearn.metrics import confusion_matrix
from    sklearn.metrics import precision_score
from    sklearn.metrics import recall_score
from    sklearn.metrics import f1_score
from    sklearn.metrics import roc_auc_score
from    sklearn.metrics import ConfusionMatrixDisplay
from    sklearn.metrics import RocCurveDisplay

In [2]:
RANDOM_SEED = np.random.seed(0)

In [3]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction


c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Test

In [4]:
df_test = pd.read_csv('data\\original\\air_system_present_year.csv',
                      encoding='utf-8',
                      na_values=['na']
                      )
df_test.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,60,0.0,20.0,12.0,0.0,0.0,0.0,0.0,0.0,...,1098.0,138.0,412.0,654.0,78.0,88.0,0.0,0.0,0.0,0.0
1,neg,82,0.0,68.0,40.0,0.0,0.0,0.0,0.0,0.0,...,1068.0,276.0,1620.0,116.0,86.0,462.0,0.0,0.0,0.0,0.0
2,neg,66002,2.0,212.0,112.0,0.0,0.0,0.0,0.0,0.0,...,495076.0,380368.0,440134.0,269556.0,1315022.0,153680.0,516.0,0.0,0.0,0.0
3,neg,59816,NaN,1010.0,936.0,0.0,0.0,0.0,0.0,0.0,...,540820.0,243270.0,483302.0,485332.0,431376.0,210074.0,281662.0,3232.0,0.0,0.0
4,neg,1814,NaN,156.0,140.0,0.0,0.0,0.0,0.0,0.0,...,7646.0,4144.0,18466.0,49782.0,3176.0,482.0,76.0,0.0,0.0,0.0


In [5]:
df_test.dtypes

class      object
aa_000      int64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [6]:
df_test.isna().sum().sort_values(ascending=False)

br_000    13129
bq_000    12981
bp_000    12721
bo_000    12376
ab_000    12363
          ...  
cj_000       86
ci_000       86
bt_000       28
aa_000        0
class         0
Length: 171, dtype: int64

In [7]:
df_test['class'].value_counts()

class
neg    15625
pos      375
Name: count, dtype: int64

In [8]:
# Calculate events proportion
total     = df_test.shape[0]
non_event = df_test[df_test['class'] == 'neg'].shape[0]
event     = df_test[df_test['class'] == 'pos'].shape[0]

non_event_percent = round(non_event / total * 100, 2)
event_percent     = round(event / total * 100, 2)

non_event_percent, event_percent

(97.66, 2.34)

In [9]:
# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_test['class'] = df_test['class'].map(map_class)

# Features to float
features = df_test.drop(columns='class').columns
df_test[features] = df_test[features].astype(float)

In [10]:
# Load features from train dataset processed
with open('train_artifacts\\top_20_features_xgb_clf.json', 'r', encoding='utf-8') as file:
    top_20_features_xgb_clf = json.load(file)

In [11]:
X_test = df_test[top_20_features_xgb_clf]
y_test = df_test['class']

# Predict

In [12]:
# Load model trained
xgb_clf = XGBClassifier(random_state=RANDOM_SEED, verbosity=0)
xgb_clf.load_model(('train_artifacts\\xgb_clf.json'))

In [13]:
# Predict event probabilities
y_predict_proba = xgb_clf.predict_proba(X_test)[:, 1]
y_predict_proba

array([0.3540612 , 0.57404876, 0.78853816, ..., 0.00410051, 0.3675424 ,
       0.7815899 ], dtype=float32)

# Metrics

In [14]:
costs = {"maintenance_cost": {"no_defect_cost": 10,
                              "preventive_cost": 25,
                              "corrective_cost": 500}
        }
no_defect_cost  = costs.get('maintenance_cost')['no_defect_cost']
preventive_cost = costs.get('maintenance_cost')['preventive_cost']
corrective_cost = costs.get('maintenance_cost')['corrective_cost']

In [15]:
results = []
for cutoff in np.arange(0.05, 0.99, 0.01):
    y_predict      = np.where(y_predict_proba >= cutoff, 1, 0)
    accuracy       = accuracy_score(y_test, y_predict)
    precision      = precision_score(y_test, y_predict, zero_division=0)
    recall         = recall_score(y_test, y_predict, zero_division=0)
    f1score        = f1_score(y_test, y_predict, zero_division=0)
    auc_value      = roc_auc_score(y_test, y_predict)
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    specificity    = tn / (tn + fp)
    no_defect_maintenance_cost  = fp * no_defect_cost
    preventive_maintenance_cost = tp * preventive_cost
    corrective_maintenance_cost = fn * corrective_cost
    total_maintenance_cost = no_defect_maintenance_cost \
                                + preventive_maintenance_cost \
                                    + corrective_maintenance_cost
    results.append([cutoff, accuracy, precision,
                    recall, f1score,
                    auc_value, tn, fp, fn, tp,
                    specificity, no_defect_maintenance_cost,
                    preventive_maintenance_cost,
                    corrective_maintenance_cost,
                    total_maintenance_cost]
                  )

In [16]:
df_metrics = pd.DataFrame(results,
                          columns=['cutoff', 'accuracy', 'precision',
                                   'recall', 'f1score' , 'auc_value',
                                   'tn', 'fp', 'fn', 'tp',
                                   'specificity', 'no_defect_maintenance_cost',
                                   'preventive_maintenance_cost',
                                   'corrective_maintenance_cost',
                                   'total_maintenance_cost']
                         )
df_metrics

,cutoff,accuracy,precision,recall,f1score,auc_value,tn,fp,fn,tp,specificity,no_defect_maintenance_cost,preventive_maintenance_cost,corrective_maintenance_cost,total_maintenance_cost
0,0.05,0.335875,0.032558,0.952000,0.062963,0.636544,5017,10608,18,357,0.321088,106080,8925,9000,124005
1,0.06,0.345438,0.032500,0.936000,0.062819,0.633632,5176,10449,24,351,0.331264,104490,8775,12000,125265
2,0.07,0.359938,0.032860,0.925333,0.063466,0.635851,5412,10213,28,347,0.346368,102130,8675,14000,124805
3,0.08,0.375937,0.033314,0.914667,0.064286,0.638837,5672,9953,32,343,0.363008,99530,8575,16000,124105
4,0.09,0.392000,0.034074,0.912000,0.065693,0.645760,5930,9695,33,342,0.379520,96950,8550,16500,122000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.94,0.976688,0.510417,0.130667,0.208068,0.563829,15578,47,326,49,0.996992,470,1225,163000,164695
90,0.95,0.977375,0.589041,0.114667,0.191964,0.556373,15595,30,332,43,0.998080,300,1075,166000,167375
91,0.96,0.978000,0.725490,0.098667,0.173709,0.548885,15611,14,338,37,0.999104,140,925,169000,170065
92,0.97,0.977875,0.744186,0.085333,0.153110,0.542315,15614,11,343,32,0.999296,110,800,171500,172410


In [23]:
df_metrics['total_maintenance_cost'].min()

np.int64(108270)

In [22]:
df_metrics.loc[df_metrics['total_maintenance_cost'].min()]

KeyError: np.int64(108270)

In [17]:
# # Area under curve
# RocCurveDisplay.from_predictions(y_test, y_predict)
# plt.show()

In [18]:
# # Confusion matrix
# ConfusionMatrixDisplay(confusion_matrix(y_test, y_predict)).plot()
# plt.show()

In [19]:
# # Confusion matrix with norm data based on true values
# ConfusionMatrixDisplay(confusion_matrix(y_test,
#                                         y_predict,
#                                         normalize='true')
#                       ).plot()
# plt.show()

In [20]:
# # Confusion matrix with norm data based on predict values
# plot_norm_pred_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix(y_test,
#                                                                             y_predict,
#                                                                             normalize='pred')
#                                                         ).plot()

In [21]:
# # Confusion matrix with norm data based on all dataset
# plot_norm_all_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix(y_test,
#                                                                             y_predict,
#                                                                             normalize='all')
#                                                        ).plot()